Pororo Extractive Summarization

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [ ]:
import numpy as np
import pandas as pd

import json
from pandas.io.json import json_normalize

import re
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

DIR = "/content/drive/MyDrive/DACON_LG/data/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

In [ ]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [ ]:
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [ ]:
train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
testcopy = test['total']

In [ ]:
testcopy

In [ ]:
!pip install --upgrade pororo

In [ ]:
!pip install kollocate

In [ ]:
!pip install koparadigm

In [ ]:
!pip install g2pk

In [ ]:
testcopy_total_list = testcopy.tolist()

In [ ]:
testcopy_total_list[0]

In [ ]:
'''
running this cell, error occurs
Change 'generations_utils.py' line 1835 from 
# next_indices = (next_tokens / vocab_size).long()
-> next_indices = next_tokens.true_divide(vocab_size).long()
'''
from pororo import Pororo

summ_ext = Pororo(task="summarization", model="extractive", lang="ko")
ext_result = []
for i in range(0,len(testcopy_total_list)):
  ext_result.append(summ_ext(testcopy_total_list[i]))

In [ ]:
ext_result[:10]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/DACON_LG/data/sample_submission.csv')

In [ ]:
submission

In [ ]:
ext_result = pd.Series(ext_result)

In [ ]:
submission['summary'] = ext_result

In [ ]:
submission.to_csv('/content/drive/MyDrive/DACON_LG/data/dacon_pororo_extract.csv', index=False)